In [1]:
import numpy as np
import torch 
import torchvision
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_0_F = np.loadtxt(open("../../data/motor_fault/train_0_F.csv","rb"), delimiter=",", skiprows=0)
train_1_F = np.loadtxt(open("../../data/motor_fault/train_1_F.csv","rb"), delimiter=",", skiprows=0)
print("train_0_F.shape: ", train_0_F.shape)
print("train_1_F.shape: ", train_1_F.shape)

train_0_F.shape:  (700000, 15)
train_1_F.shape:  (42000, 15)


In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

In [4]:
# Hyper-parameters
input_size = 14
hidden_size1 = 100
hidden_size2 = 50
num_classes = 2
num_epochs = 5
batch_size = 100
learning_rate = 1e-3
valid_size = 0.2

In [5]:
# Dataset
train_F = np.concatenate((train_0_F, train_1_F), axis=0)
x = torch.tensor(train_F[:,:-1], dtype=torch.float32)
y = torch.tensor(train_F[:,-1], dtype=torch.long)
dataset = torch.utils.data.TensorDataset(x, y)

num_train = len(dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
np.random.shuffle(indices)
train_idx, test_idx = indices[:split], indices[split:]
trainset = torch.utils.data.Subset(dataset=dataset, indices=train_idx)
testset = torch.utils.data.Subset(dataset=dataset, indices=test_idx)

trainloader = torch.utils.data.DataLoader(trainset,
                                     batch_size = batch_size,
                                     shuffle = True,
                                     num_workers = 2)
testloader = torch.utils.data.DataLoader(testset,
                                     batch_size = batch_size,
                                     shuffle = False,
                                     num_workers = 2)


In [6]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(NeuralNet,self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2,num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [7]:
model = NeuralNet(input_size, hidden_size1, hidden_size2, num_classes).to(device)
weight=torch.tensor([30,500]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

In [8]:
def test():
    # test on all test data
    correct = 0.0
    total = 0.0
    model.eval()
    with torch.no_grad():
        for x, y in testloader:
            x = x.to(device)
            y = y.to(device)
            outputs = model(x)
            predicted = torch.max(outputs.data, 1)[1]
            total += y.size(0)
            correct += (predicted == y).sum()

    model.train()
    print('Test_acc: {:.2f}%'.format(100 * correct / total))

1-3/(50+3) = 0.943

In [9]:
test()

Test_acc: 26.00%


In [10]:
# Train the model 

total_step = len(trainloader)
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (x, y) in enumerate(trainloader):
        # Move tensors to the configured device
        x = x.to(device)
        y = y.to(device)
        
        #print("batch_x: ", type(x), x.size())
        #print("batch_y: ", type(y), y.size())
        #print(x)
        #print(y)
        
        # Forward pass
        outputs = model(x)
        #print(outputs)
        loss = criterion(outputs,y)
        
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() 
        if (i+1) % 100 == 0:
            print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, running_loss/100))
            running_loss = 0.0
    test()

Epoch: [1/5], Step: [100/1484], Loss: 0.2330
Epoch: [1/5], Step: [200/1484], Loss: 0.2106
Epoch: [1/5], Step: [300/1484], Loss: 0.1950
Epoch: [1/5], Step: [400/1484], Loss: 0.1978
Epoch: [1/5], Step: [500/1484], Loss: 0.1731
Epoch: [1/5], Step: [600/1484], Loss: 0.1547
Epoch: [1/5], Step: [700/1484], Loss: 0.1592
Epoch: [1/5], Step: [800/1484], Loss: 0.1473
Epoch: [1/5], Step: [900/1484], Loss: 0.1410
Epoch: [1/5], Step: [1000/1484], Loss: 0.1292
Epoch: [1/5], Step: [1100/1484], Loss: 0.1282
Epoch: [1/5], Step: [1200/1484], Loss: 0.1240
Epoch: [1/5], Step: [1300/1484], Loss: 0.1313
Epoch: [1/5], Step: [1400/1484], Loss: 0.1232
Test_acc: 96.00%
Epoch: [2/5], Step: [100/1484], Loss: 0.1186
Epoch: [2/5], Step: [200/1484], Loss: 0.1118
Epoch: [2/5], Step: [300/1484], Loss: 0.1183
Epoch: [2/5], Step: [400/1484], Loss: 0.1131
Epoch: [2/5], Step: [500/1484], Loss: 0.1099
Epoch: [2/5], Step: [600/1484], Loss: 0.1091
Epoch: [2/5], Step: [700/1484], Loss: 0.1036
Epoch: [2/5], Step: [800/1484], L